In [5]:
import os

# Check these before running
# KPUM Workstation
sourcedataparentfolder = '/mnt/d/Finn/KPUM_NODDI/Data'
sourcedatabasefolder = 'sourcedata'
studydir = '/mnt/d/Finn/KPUM_NODDI/Data'
niftibasefolder = 'nifti' # nifti basefolder to go within studydir

# Finn's Laptop
sourcedataparentfolder = '/mnt/d/Finn/KPUM_NODDI/Data'
sourcedatabasefolder = 'sourcedata' # nifti basefolder to go within sourcedataparentfolder
studydir = '/mnt/s/Research/Projects/KPUM_NODDI/Data'
niftibasefolder = 'nifti' # nifti basefolder to go within studydir

# tsv-files to keep track of in studydir/niftibasefolder
subjecttrackerpath = os.path.join(studydir, niftibasefolder)
subjecttrackerfile = 'Subject_Tracker_for_sourcedata2nifti.tsv'

###################################################################################
# USER INPUT

# Participant details
subject = '035'
session = 'MR1'

# Define good DCM datafolder as input (and if no data is not good enough or does not exist, leave blank)

## ANATOMY
# FLAIR
flair = 's03_FLAIR_TRA_FS'
# T2w Tra
t2wtra = 's04_T2W_TRA'
# High-resolution T1w MPRAGE
t1w_mprage = 's19_MPRAGE'
# T2star
t2star = 's07_T2starW_COR'
# SWI (if available)
swi = 's09_SWI_TRA'
# SWI minIP (if available)
swiminip = 's08_SWI_TRA'
# High-resolution T2w MCRIB (if available)
t2w_mcrib = 's05_T2W_MCRIB'

## DIFFUSION
# DKI dir-AP data
dki_dirAP = 's17_DKI_2.5mm_NODDI_dir-AP_79dir_0_1000_2000'
# DKI dir-PA data (if available)
dki_dirPA = 's16_DKI_2.5mm_NODDI_dir-PA_2dir_0'


###################################################################################



In [6]:

# Define local functions
def dicom2nifti(inputdicomdir, outputniftidir, outfilebase):
    # Convert DCMs to NIfTIs using dcm2niix
    import subprocess
    from datetime import datetime
    
    print("Start converting DCMs")
    starttime=datetime.now()
    p=subprocess.Popen(f"dcm2niix -a y -d 1 -b y -w 1 -o {outputniftidir} -f {outfilebase} {inputdicomdir}", stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()  
    #This makes the wait possible
    p_status = p.wait()
    #This will give you the output of the command being executed
    #print(f"Command output: {output}")
    endtime=datetime.now()
    print("Done converting DCMs. Elapsed time:", endtime-starttime)


# Create conversion file sourcedata2nifti.tsv
import os, subprocess
import pandas as pd

# Define I/O folders and files
sourcedatafolder = os.path.join(sourcedatabasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
sourcedatainputfolder = os.path.join(sourcedataparentfolder, sourcedatafolder)
niftifolder = os.path.join(niftibasefolder, os.path.join(f'sub-{subject}',f'ses-{session}'))
niftioutputfolder = os.path.join(studydir, niftifolder)
if not os.path.exists(niftioutputfolder): # then make this directory
    os.makedirs(niftioutputfolder)
subjecttrackertsv = os.path.join(subjecttrackerpath, subjecttrackerfile)

# Create list for matching file transfers (DO NOT CHANGE)
sourcedatalist = [t1w_mprage, 
                  t2w_mcrib,
                  t2wtra,
                  flair,
                  t2star,
                  swi,
                  swiminip,
                  dki_dirAP,
                  dki_dirPA ] 
niftidatalist = [os.path.join('anat',f'sub-{subject}_ses-{session}_acq-mprage_T1w.nii'), 
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-mcrib_T2w.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-tra_T2w.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_FLAIR.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-t2star_T2starw.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-swi_T2starw.nii'),
                 os.path.join('anat',f'sub-{subject}_ses-{session}_acq-swi_rec-mnip_T2starw.nii'),
                 os.path.join('dwi',f'sub-{subject}_ses-{session}_dir-AP_dwi.nii'),
                 os.path.join('fmap',f'sub-{subject}_ses-{session}_dir-PA_epi.nii'),
                 ]
# and make this into a DataFrame
sample = {'input': sourcedatalist, 
          'output': niftidatalist,
          'converted': '',
          'comments': ''}
df = pd.DataFrame(sample)

# Locate only the files that are going to be transferred and set this entry to Pending (i.e. not have entry '' in sourcedatalist)
df.loc[df['input'] != '', ['converted']] = 'Pending'
# and save to a LOCAL tsv-file in niftioutputfolder
df.loc[df['converted'] == 'Pending'].to_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t", index=False)
# and read df again from the newly written file
df=pd.read_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t")
df.fillna('', inplace=True)

# Now loop over these Pending entries in df and perform conversion using dcm2niix
for ind, row in df.loc[df['converted'] == 'Pending'].iterrows() :
    rowdata = row['input']
    inputdicomdir = os.path.join(sourcedatainputfolder, df['input'][ind])
    outfilebase = os.path.splitext(df['output'][ind])[0] # workaround to get rid of file-extension
    print('Converting DCMs in', inputdicomdir ,'and save in', os.path.join(niftioutputfolder, outfilebase))
    # Convert DCMs using local function 
    dicom2nifti(inputdicomdir, niftioutputfolder, outfilebase)
     
    # Update tracker file by writing 'Done'
    df.at[ind, 'converted'] = 'Done'
    df.to_csv(os.path.join(niftioutputfolder, f'sub-{subject}_ses-{session}_sourcedata2nifti.tsv'), sep="\t", index=False)
        
    print('Finished converting DCMs in', inputdicomdir,'and save in', os.path.join(niftioutputfolder, outfilebase))
    print()
        
# Finally, update subjecttrackerfile 
df = pd.read_csv(subjecttrackertsv, sep="\t")
df.fillna('', inplace=True)
# Check if sourcedatafolder is an entry in df, and if not add it to the file
if not df['input'].isin([sourcedatafolder]).any() : 
    # We should add as new entry in the bottom.
    new_row = {'input': sourcedatafolder, 
               'output': niftifolder, 
               'converted': 'Done', 
               'QC': 'Pending', 
               'comments': ''} 
    df.loc[len(df)] = new_row
    # then write to subjecttrackertsv-file
    print(f"{sourcedatafolder} is not in {subjecttrackerfile}. Adding as new entry")
else:
    # update entry/ies 
    print(f"The sub-{subject}_ses-{session} has an entry {sourcedatafolder} is in {subjecttrackerfile}. Updating the rest of the entries")
    df.loc[df['input'] == sourcedatafolder, ['output']] = niftifolder
    df.loc[df['input'] == sourcedatafolder, ['converted']] = 'Done'
    df.loc[df['input'] == sourcedatafolder, ['QC']] = 'Pending'
    df.loc[df['input'] == sourcedatafolder, ['comments']] = ''
# sort the rows according to 'input'
df = df.sort_values(by = 'input')
# and write to subjecttrackertsv-file
df.to_csv(subjecttrackertsv, sep="\t", index=False)

Converting DCMs in /mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI/sourcedata/sub-035/ses-MR1/s19_MPRAGE and save in /mnt/s/Research/Projects/KPUM_NODDI/Data/nifti/sub-035/ses-MR1/anat/sub-035_ses-MR1_acq-mprage_T1w
Start converting DCMs
Done converting DCMs. Elapsed time: 0:00:10.135618
Finished converting DCMs in /mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI/sourcedata/sub-035/ses-MR1/s19_MPRAGE and save in /mnt/s/Research/Projects/KPUM_NODDI/Data/nifti/sub-035/ses-MR1/anat/sub-035_ses-MR1_acq-mprage_T1w

Converting DCMs in /mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI/sourcedata/sub-035/ses-MR1/s05_T2W_MCRIB and save in /mnt/s/Research/Projects/KPUM_NODDI/Data/nifti/sub-035/ses-MR1/anat/sub-035_ses-MR1_acq-mcrib_T2w
Start converting DCMs
Done converting DCMs. Elapsed time: 0:00:10.738281
Finished converting DCMs in /mnt/d/Finn/KPUM_NODDI/DICOM_KPUM_NODDI/sourcedata/sub-035/ses-MR1/s05_T2W_MCRIB and save in /mnt/s/Research/Projects/KPUM_NODDI/Data/nifti/sub-035/ses-MR1/anat/sub-035_ses-MR1_acq-mcrib_T2w